Original question: https://discourse.pymc.io/t/11-16-rethinking-code/5181

# Introduction
Contains the observed data and the sampling of the PyMC3 model

In [1]:
import arviz as az
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pymc3 as pm
import theano.tensor as tt
import warnings

from scipy import stats
from scipy.special import expit as logistic
from scipy.special import softmax

warnings.simplefilter(action="ignore", category=FutureWarning)
RANDOM_SEED = 8927
np.random.seed(286)

In [2]:
az.style.use("arviz-darkgrid")
az.rcParams["stats.hdi_prob"] = 0.89 # name was changed in latest ArviZ

In [3]:
d = pd.read_csv(
    "https://raw.githubusercontent.com/pymc-devs/resources/master/Rethinking_2/Data/chimpanzees.csv", 
    sep=";"
)
# we change "actor" to zero-index
d.actor = d.actor - 1
d["treatment"] = d.prosoc_left + 2 * d.condition
d[["actor", "prosoc_left", "condition", "treatment"]]

,actor,prosoc_left,condition,treatment
0,0,0,0,0
1,0,0,0,0
2,0,1,0,1
3,0,0,0,0
4,0,1,0,1
...,...,...,...,...
499,6,1,1,3
500,6,1,1,3
501,6,0,1,2
502,6,0,1,2


In [4]:
actor_idx, actors = pd.factorize(d.actor)
treat_idx, treatments = pd.factorize(d.treatment)

In [5]:
with pm.Model() as m11_4:
    a = pm.Normal("a", 0.0, 1.5, shape=len(actors))
    b = pm.Normal("b", 0.0, 0.5, shape=len(treatments))

    actor_id = pm.intX(pm.Data("actor_id", actor_idx))
    treat_id = pm.intX(pm.Data("treat_id", treat_idx))
    p = pm.Deterministic("p", pm.math.invlogit(a[actor_id] + b[treat_id]))

    pulled_left = pm.Binomial("pulled_left", 1, p, observed=d.pulled_left)

    trace_11_4 = pm.sample(random_seed=RANDOM_SEED)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [b, a]
Sampling 4 chains, 0 divergences: 100%|██████████| 4000/4000 [00:01<00:00, 2388.27draws/s]


In [6]:
treatment_names = [f"treatment {i+1}" for i in treatments]
# I have given some names to the treatments to show ArviZ capabilities with 
# labels thanks to xarray
dims = {
    "a": ["actor"], 
    "b": ["treatment"],
    "actor_id": ["experiment"],
    "treat_id": ["experiment"],
    "p": ["experiment"],
    "pulled_left": ["experiment"]
}
coords = {"actor": actors, "treatment": treatment_names} 

# also, it is highly recommended to either pass the model instance to `from_pymc3`
# or to call it from the corresponding model context
idata_11_4 = az.from_pymc3(trace_11_4, model=m11_4, dims=dims, coords=coords)
az.summary(idata_11_4, var_names=["a", "b"], round_to=2)

,mean,sd,hdi_5.5%,hdi_94.5%,mcse_mean,mcse_sd,ess_mean,ess_sd,ess_bulk,ess_tail,r_hat
a[0],-0.44,0.32,-0.97,0.05,0.01,0.01,696.64,696.64,697.77,1204.46,1.01
a[1],3.90,0.76,2.82,5.27,0.02,0.01,1685.56,1590.31,1734.73,1307.42,1.00
a[2],-0.75,0.33,-1.31,-0.25,0.01,0.01,684.50,684.50,684.25,951.56,1.01
a[3],-0.74,0.33,-1.28,-0.24,0.01,0.01,746.43,746.43,758.53,1041.13,1.01
a[4],-0.44,0.33,-0.92,0.11,0.01,0.01,672.00,672.00,673.09,1275.21,1.00
a[5],0.49,0.34,-0.03,1.03,0.01,0.01,623.04,623.04,622.09,1042.27,1.01
a[6],1.95,0.41,1.34,2.65,0.01,0.01,978.12,974.13,995.21,1331.11,1.00
b[treatment 1],-0.04,0.29,-0.53,0.38,0.01,0.01,591.45,591.45,590.00,1137.32,1.01
b[treatment 2],0.48,0.29,0.03,0.95,0.01,0.01,637.00,637.00,637.81,1418.87,1.01
b[treatment 3],-0.39,0.28,-0.84,0.05,0.01,0.01,596.72,596.72,595.32,989.79,1.01


_Note: I am using this notebook also as a test for [arviz-devs/arviz#1201](https://github.com/arviz-devs/arviz/pull/1201) so I have coordinate values instead of indexes shown in the summary. This has no relation to the current quesion, it is only to prevent surprises when running the notebook on ArviZ<=0.8.3_

In [7]:
idata_11_4.posterior["p"].shape

(4, 500, 504)

We have 504 different $p$ but $n_{actors} \times n_{treatments} = 28$?!?! How do we get the 28 $p$ corresponding the each `actor`-`treatment` combination?

# To the question at hand
As we have just seen, our values for $p$ correspond to each experiment, and there are 504 of them. 

Each experiment was done for a specific combination of `actor` and `treatment` and different experiments correspond to the same combination of `actor` and `treatment`. Thus, to get the $p$ corresponding to each of the 28 `actor`-`treatment` combinations we have (at least) 3 options (we'll focus on the 2nd and 3dr ones):

1. Group-by and average the original 504 $p$. This can be straighforward if there are the same number of experiments for each `actor`-`treatment` (a reshape could be enough if properly ordered) but with unsorted and ragged data things can get out of control.
1. `pm.sample_posterior_predictive`: [`pm.Data`](https://docs.pymc.io/api/model.html#pymc3.model.set_data) containers are thought to allow changing the data in the model and get predictions for new values for example. In this particular case, we are modifying the data to contain only one experiment per `actor`-`treatment` combination which automatically gets us to the case in option 1 where only a reshape is needed.
3. Calculate $p$ manually from $a$ and $b$. We can manually calculate $logistic(a+b)$ for every combination of `actor`-`treatment`

It should be noted that even though we are using `pm.sample_posterior_predictive` in option 2, we are not getting samples from the posterior predictive. We are using it to cleverly reevaluate a deterministic function from the posterior. 

We would be sampling from the posterior predictive if we obtained values from `pulled_left`. The obtained values could then be averaged to _estimate_ the probability, but given that we do know the probability it does not make much sense to _estimate_ it.


## `pm.sample_posterior_predictive` version

In [8]:
with m11_4:
    pm.set_data({"actor_id": np.repeat(range(7), 4), "treat_id": list(range(4)) * 7})
    p_post = pm.sample_posterior_predictive(
        trace_11_4, random_seed=RANDOM_SEED, var_names=["p"]
    )["p"]
p_mu = p_post.mean(0).reshape((7, 4))

100%|██████████| 2000/2000 [00:00<00:00, 3839.00it/s]


In [9]:
p_mu

array([[0.38273298, 0.50794882, 0.3064966 , 0.48028027],
       [0.97383758, 0.9842027 , 0.96366044, 0.98230814],
       [0.31552197, 0.43399235, 0.24723306, 0.40704301],
       [0.31658471, 0.43514105, 0.24811081, 0.40830217],
       [0.38371248, 0.50898154, 0.30735674, 0.4813107 ],
       [0.60644722, 0.71919498, 0.52349316, 0.69665124],
       [0.8650691 , 0.91412716, 0.82050517, 0.90508358]])

## Manual version
The manual version will use ArviZ and xarray to showcase its capabilities. The `posterior` group in the `InferenceData` object is an `xarray.Dataset`. As it can be seen below, the dataset stores both the data and its dimensions and coordinate values:

In [10]:
idata_11_4.posterior

<xarray.Dataset>
Dimensions:     (actor: 7, chain: 4, draw: 500, experiment: 504, treatment: 4)
Coordinates:
  * chain       (chain) int64 0 1 2 3
  * draw        (draw) int64 0 1 2 3 4 5 6 7 ... 492 493 494 495 496 497 498 499
  * actor       (actor) int64 0 1 2 3 4 5 6
  * treatment   (treatment) <U11 'treatment 1' 'treatment 2' ... 'treatment 4'
  * experiment  (experiment) int64 0 1 2 3 4 5 6 ... 497 498 499 500 501 502 503
Data variables:
    a           (chain, draw, actor) float64 0.135 3.439 ... 0.8038 2.469
    b           (chain, draw, treatment) float64 -0.3593 0.08189 ... 0.3017
    p           (chain, draw, experiment) float64 0.4442 0.4442 ... 0.8828
Attributes:
    created_at:                 2020-06-04T22:45:38.724507
    arviz_version:              0.8.3
    inference_library:          pymc3
    inference_library_version:  3.8

As dimensions are named, `xarray` is able to automatically broadcast, here we are adding two 3d arrays with _different_ shapes without any problem and getting the desired 4d output

In [11]:
p_mu_samples = logistic(idata_11_4.posterior["a"]+ idata_11_4.posterior["b"])
p_mu_samples

<xarray.DataArray (chain: 4, draw: 500, actor: 7, treatment: 4)>
array([[[[0.4441712 , 0.55402312, 0.3159088 , 0.45416202],
         [0.95606827, 0.97129007, 0.92634161, 0.95773349],
         [0.38020658, 0.48813174, 0.26171787, 0.38976755],
         ...,
         [0.42020547, 0.5297796 , 0.29518825, 0.43007432],
         [0.68420601, 0.77106923, 0.55595995, 0.69286569],
         [0.88616883, 0.92367647, 0.81814109, 0.89017923]],

        [[0.32700514, 0.46045575, 0.26880136, 0.50494431],
         [0.99531797, 0.99732888, 0.99382087, 0.99776409],
         [0.26036758, 0.38206201, 0.21031801, 0.42494175],
         ...,
         [0.33704464, 0.47172058, 0.27779151, 0.51625612],
         [0.58231819, 0.71003434, 0.51333344, 0.74532588],
         [0.90127602, 0.94129532, 0.87352952, 0.95040619]],

        [[0.25821549, 0.37166134, 0.25319262, 0.38688608],
         [0.90224481, 0.94005933, 0.89989218, 0.94360161],
         [0.3627958 , 0.49173028, 0.3567169 , 0.5078978 ],
         ...,
         [0.37010869, 0.49960439, 0.36397725, 0.51576809],
         [0.57781228, 0.69930008, 0.5713611 , 0.71272237],
         [0.79549626, 0.86858989, 0.7911692 , 0.87579774]],

        ...,

        [[0.41141284, 0.5076597 , 0.34369729, 0.61522334],
         [0.97425583, 0.98240233, 0.96593192, 0.98858014],
         [0.31092585, 0.39962558, 0.25265019, 0.50791326],
         ...,
         [0.37596846, 0.47055246, 0.31100422, 0.57950811],
         [0.54872531, 0.64205411, 0.47671404, 0.73555108],
         [0.82702274, 0.87582137, 0.78175772, 0.9162246 ]],

        [[0.34860267, 0.44440894, 0.25799757, 0.53637973],
         [0.98473185, 0.989733  , 0.97669224, 0.99287902],
         [0.3371221 , 0.431865  , 0.2483632 , 0.52368672],
         ...,
         [0.38896937, 0.48756585, 0.29258508, 0.5791578 ],
         [0.54462626, 0.64127009, 0.43727369, 0.72110408],
         [0.86547961, 0.90580591, 0.80695625, 0.93292613]],

        [[0.27939853, 0.56563795, 0.20965545, 0.493642  ],
         [0.95627397, 0.98656842, 0.93735302, 0.98213902],
         [0.20327621, 0.46147166, 0.14861582, 0.39080524],
         ...,
         [0.32762106, 0.62070895, 0.25001713, 0.55058863],
         [0.49769836, 0.76893658, 0.4040151 , 0.71357424],
         [0.90461133, 0.96955952, 0.86645702, 0.95974986]]],


       [[[0.29230878, 0.49718931, 0.24311653, 0.5228141 ],
         [0.9521296 , 0.97943043, 0.93927385, 0.98139822],
         [0.25408821, 0.44918316, 0.20942462, 0.47466948],
         ...,
         [0.24656325, 0.43928306, 0.20286266, 0.46468154],
         [0.59373307, 0.7777105 , 0.53194348, 0.79493551],
         [0.79660123, 0.9036226 , 0.75282097, 0.91219239]],

        [[0.36635053, 0.50596793, 0.30590282, 0.5609953 ],
         [0.9588454 , 0.9763434 , 0.9466954 , 0.98095088],
         [0.31199312, 0.44545801, 0.25687839, 0.5005725 ],
         ...,
         [0.28453008, 0.41330423, 0.23262644, 0.46779642],
         [0.61241248, 0.73676902, 0.54637264, 0.77739888],
         [0.85471835, 0.91244622, 0.81767268, 0.92858841]],

        [[0.3837187 , 0.61631435, 0.26100551, 0.55675367],
         [0.96468772, 0.98600969, 0.93938132, 0.98217845],
         [0.26720625, 0.48472571, 0.17139132, 0.42383437],
         ...,
         [0.29825557, 0.52301093, 0.19425843, 0.46161878],
         [0.75254097, 0.88694793, 0.63303408, 0.85984479],
         [0.85016375, 0.93605275, 0.7629517 , 0.91965561]],

        ...,

        [[0.41093853, 0.471524  , 0.23719419, 0.36942022],
         [0.97972067, 0.98407364, 0.9556224 , 0.97594463],
         [0.5141998 , 0.57514494, 0.3205549 , 0.47058315],
         ...,
         [0.33074744, 0.38728313, 0.18051772, 0.29329707],
         [0.78581351, 0.8243254 , 0.62053862, 0.75496207],
         [0.8424088 , 0.87239699, 0.70437595, 0.81781942]],

        [[0.27873963, 0.49098128, 0.31845388, 0.48612488],
         [0.98659982, 0.99458762, 0.98889104, 0.99448198],
         [0.2111228 , 0.40046514, 0.24446853, 0.39580787],
         ...,
         [0.39

We can also use dimension _names_ to indicate the dimensions along which the mean should be taken

In [12]:
p_mu_az = p_mu_samples.mean(dim=("chain", "draw"))
p_mu_az

<xarray.DataArray (actor: 7, treatment: 4)>
array([[0.38273298, 0.50794882, 0.3064966 , 0.48028027],
       [0.97383758, 0.9842027 , 0.96366044, 0.98230814],
       [0.31552197, 0.43399235, 0.24723306, 0.40704301],
       [0.31658471, 0.43514105, 0.24811081, 0.40830217],
       [0.38371248, 0.50898154, 0.30735674, 0.4813107 ],
       [0.60644722, 0.71919498, 0.52349316, 0.69665124],
       [0.8650691 , 0.91412716, 0.82050517, 0.90508358]])
Coordinates:
  * actor      (actor) int64 0 1 2 3 4 5 6
  * treatment  (treatment) <U11 'treatment 1' 'treatment 2' ... 'treatment 4'

Note how thanks to xarray, this option ends up being a one liner:

    logistic(idata_11_4.posterior["a"]+ idata_11_4.posterior["b"]).mean(dim=("chain", "draw"))

It has been extended to show the intermediate steps.

## Compare results

In [13]:
np.allclose(p_mu, p_mu_az.values) # compare means

True

In [14]:
np.allclose(p_post.reshape((4,-1,7,4)), p_mu_samples) # compare all samples, requires reshaping p_post

True

We can see that here there is actually NO random sampling involved and that both approaches return the same result up to machine error. To be extra sure, go back to the `pm.sample_posterior_predictive` call and change the random seed to see we still get the same result.

## Groupby `p` in `trace_11_4`
Finally we'll consider the first option, due to its relation to the following question:

> Am I correct thinking that since p is a deterministic function of the parameters a and b , there should be a value of p in trace_11_4 for every value returned by sample_posterior_predictive?

We will explore the values stored in `p` in `trace_11_4` (or what is the same, in `idata_11_4`). `p` is defined in the following line:

    p = pm.Deterministic("p", pm.math.invlogit(a[actor_id] + b[treat_id]))

Thus, at every sample, `p` will contain only 28 different values, these unique values will be equal to the values returned by `pm.sample_posterior_predictive` and will be equal to the values stored in `p_mu_samples`. To check this, we'll compare the values of `p` in each sample for a couple of `actor`-`treatment` combination with the corresponding values in the other alternatives. 

In [15]:
def get_mask(actor, treatment):
    return (actor == actor_idx) & (treatment == treat_idx)

Example 1: actor 2 and treatment 1

In [16]:
mask_2_1 = get_mask(actor=2, treatment=1)

In [17]:
da_2_1 = idata_11_4.posterior.p.sel(experiment=mask_2_1) 
da_2_1 # the database icon can be used to explore the data

<xarray.DataArray 'p' (chain: 4, draw: 500, experiment: 18)>
array([[[0.48813174, 0.48813174, 0.48813174, ..., 0.48813174,
         0.48813174, 0.48813174],
        [0.38206201, 0.38206201, 0.38206201, ..., 0.38206201,
         0.38206201, 0.38206201],
        [0.49173028, 0.49173028, 0.49173028, ..., 0.49173028,
         0.49173028, 0.49173028],
        ...,
        [0.39962558, 0.39962558, 0.39962558, ..., 0.39962558,
         0.39962558, 0.39962558],
        [0.431865  , 0.431865  , 0.431865  , ..., 0.431865  ,
         0.431865  , 0.431865  ],
        [0.46147166, 0.46147166, 0.46147166, ..., 0.46147166,
         0.46147166, 0.46147166]],

       [[0.44918316, 0.44918316, 0.44918316, ..., 0.44918316,
         0.44918316, 0.44918316],
        [0.44545801, 0.44545801, 0.44545801, ..., 0.44545801,
         0.44545801, 0.44545801],
        [0.48472571, 0.48472571, 0.48472571, ..., 0.48472571,
         0.48472571, 0.48472571],
        ...,
        [0.57514494, 0.57514494, 0.57514494, ..., 0.57514494,
         0.57514494, 0.57514494],
        [0.40046514, 0.40046514, 0.40046514, ..., 0.40046514,
         0.40046514, 0.40046514],
        [0.46975951, 0.46975951, 0.46975951, ..., 0.46975951,
         0.46975951, 0.46975951]],

       [[0.46407502, 0.46407502, 0.46407502, ..., 0.46407502,
         0.46407502, 0.46407502],
        [0.35424883, 0.35424883, 0.35424883, ..., 0.35424883,
         0.35424883, 0.35424883],
        [0.60674802, 0.60674802, 0.60674802, ..., 0.60674802,
         0.60674802, 0.60674802],
        ...,
        [0.6003085 , 0.6003085 , 0.6003085 , ..., 0.6003085 ,
         0.6003085 , 0.6003085 ],
        [0.56894664, 0.56894664, 0.56894664, ..., 0.56894664,
         0.56894664, 0.56894664],
        [0.29732839, 0.29732839, 0.29732839, ..., 0.29732839,
         0.29732839, 0.29732839]],

       [[0.55170335, 0.55170335, 0.55170335, ..., 0.55170335,
         0.55170335, 0.55170335],
        [0.38741005, 0.38741005, 0.38741005, ..., 0.38741005,
         0.38741005, 0.38741005],
        [0.35868059, 0.35868059, 0.35868059, ..., 0.35868059,
         0.35868059, 0.35868059],
        ...,
        [0.51119935, 0.51119935, 0.51119935, ..., 0.51119935,
         0.51119935, 0.51119935],
        [0.4345792 , 0.4345792 , 0.4345792 , ..., 0.4345792 ,
         0.4345792 , 0.4345792 ],
        [0.50228659, 0.50228659, 0.50228659, ..., 0.50228659,
         0.50228659, 0.50228659]]])
Coordinates:
  * chain       (chain) int64 0 1 2 3
  * draw        (draw) int64 0 1 2 3 4 5 6 7 ... 492 493 494 495 496 497 498 499
  * experiment  (experiment) int64 145 146 147 150 152 ... 170 173 176 177 179

In [18]:
np.allclose(da_2_1.values, p_post.reshape((4,-1,7,4))[:,:,2,1,None])

True

In [19]:
np.allclose(da_2_1.values, p_mu_samples.isel(actor=2, treatment=1).values[:,:,None])

True

_As shown in this example, positional indexing, `isel`, can still be used with xarray. Below we use label based indexing, `sel`_

Example 1: actor 5 and treatment 3

In [20]:
mask_5_3 = get_mask(actor=5, treatment=3)
da_5_3 = idata_11_4.posterior.p.sel(experiment=mask_5_3) 
print(np.allclose(da_5_3.values, p_post.reshape((4,-1,7,4))[:,:,5,3,None]))
print(np.allclose(da_5_3.values, p_mu_samples.sel(actor=5, treatment="treatment 4").values[:,:,None]))

True
True


## A mixed pandas-xarray implementation of option 1

In [21]:
d["repetition"] = d.groupby(["actor", "treatment"]).cumcount()
d

,actor,recipient,condition,block,trial,prosoc_left,chose_prosoc,pulled_left,treatment,repetition
0,0,NaN,0,1,2,0,1,0,0,0
1,0,NaN,0,1,4,0,0,1,0,1
2,0,NaN,0,1,6,1,0,0,1,0
3,0,NaN,0,1,8,0,1,0,0,2
4,0,NaN,0,1,10,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...
499,6,4.0,1,6,64,1,1,1,3,16
500,6,6.0,1,6,66,1,1,1,3,17
501,6,3.0,1,6,68,0,0,1,2,15
502,6,7.0,1,6,70,0,0,1,2,16


In [22]:
aux_names = {"actor": "actor_aux", "treatment": "treatment_aux"}
aux_names_inv = {v: k for k, v in aux_names.items()}
multi_idx = pd.MultiIndex.from_frame(
    d[["actor", "treatment", "repetition"]].rename(columns=aux_names)
)
idata_11_4.posterior["experiment"] = multi_idx
idata_11_4.posterior = (
    idata_11_4.posterior.unstack(
        "experiment"
    ).reset_index(
        tuple(aux_names.values()), drop=True
    ).rename(aux_names_inv)
)
idata_11_4.posterior

<xarray.Dataset>
Dimensions:     (actor: 7, chain: 4, draw: 500, repetition: 18, treatment: 4)
Coordinates:
  * chain       (chain) int64 0 1 2 3
  * draw        (draw) int64 0 1 2 3 4 5 6 7 ... 492 493 494 495 496 497 498 499
  * actor       (actor) int64 0 1 2 3 4 5 6
  * treatment   (treatment) <U11 'treatment 1' 'treatment 2' ... 'treatment 4'
  * repetition  (repetition) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17
Data variables:
    a           (chain, draw, actor) float64 0.135 3.439 ... 0.8038 2.469
    b           (chain, draw, treatment) float64 -0.3593 0.08189 ... 0.3017
    p           (chain, draw, actor, treatment, repetition) float64 0.4442 ... 0.9411
Attributes:
    created_at:                 2020-06-04T22:45:38.724507
    arviz_version:              0.8.3
    inference_library:          pymc3
    inference_library_version:  3.8

The code combines two stackoverflow answers: [reshaping tricks](https://stackoverflow.com/questions/59504320/how-do-i-subdivide-refine-a-dimension-in-an-xarray-dataset/59685729#59685729) and [changing indexing dimensions](https://stackoverflow.com/questions/61827061/xarray-make-two-dataarrays-in-the-same-dataset-use-the-same-coordinate-system/61859538#61859538). They are quite detailed and should help understanding what are the different steps doing. 

---

With ArviZ>=0.8.0, we can also automatically reshape several groups at once. For instance, observed and constant data or log_likelihood could potentially benefit from the reshape too, so here we go. We start with

In [23]:
idata_11_4.log_likelihood

<xarray.Dataset>
Dimensions:      (chain: 4, draw: 500, experiment: 504)
Coordinates:
  * chain        (chain) int64 0 1 2 3
  * draw         (draw) int64 0 1 2 3 4 5 6 7 ... 493 494 495 496 497 498 499
  * experiment   (experiment) int64 0 1 2 3 4 5 6 ... 498 499 500 501 502 503
Data variables:
    pulled_left  (chain, draw, experiment) float64 -0.5873 -0.8115 ... -0.1247
Attributes:
    created_at:                 2020-06-04T22:45:38.960981
    arviz_version:              0.8.3
    inference_library:          pymc3
    inference_library_version:  3.8

In [24]:
# these do not have actor and treatment dims so we can skip all crazy renaming
def reshape_experiment_dim(ds, multi_idx):
    ds["experiment"] = multi_idx
    return ds.unstack("experiment")

In [25]:
multi_idx = pd.MultiIndex.from_frame(
    d[["actor", "treatment", "repetition"]] # no renamig here either
)
idata_11_4 = idata_11_4.map(
    reshape_experiment_dim, 
    groups=("constant_data", "observed_data", "log_likelihood"), 
    args=(multi_idx,)
)

In [26]:
idata_11_4.log_likelihood

<xarray.Dataset>
Dimensions:      (actor: 7, chain: 4, draw: 500, repetition: 18, treatment: 4)
Coordinates:
  * chain        (chain) int64 0 1 2 3
  * draw         (draw) int64 0 1 2 3 4 5 6 7 ... 493 494 495 496 497 498 499
  * actor        (actor) int64 0 1 2 3 4 5 6
  * treatment    (treatment) int64 0 1 2 3
  * repetition   (repetition) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17
Data variables:
    pulled_left  (chain, draw, actor, treatment, repetition) float64 -0.5873 ... -0.06073
Attributes:
    created_at:                 2020-06-04T22:45:38.960981
    arviz_version:              0.8.3
    inference_library:          pymc3
    inference_library_version:  3.8